In [ ]:
library(randomForest)
library(leaps)
library(tidyverse)
library(glmnet)
library(pls)
library(dbplyr)
library(ggplot2)
library(caret)
library(Metrics)
library(e1071)
library(doParallel)
library(ranger)

# Loading Data

In [ ]:
train<-read.csv("/kaggle/input/the-price-of-art/train.csv")
test<-read.csv("/kaggle/input/the-price-of-art/test.csv")
sample<-read.csv("/kaggle/input/the-price-of-art/sample.csv")

In [ ]:
head(train)
dim(train)

# EDA and Cleaning Data

In looking at the data, it seems we have 16 columns, with 97571 observations. The character columns have a lot of information, so its difficult to know where to begin with these columsn and their impact on sale value. Some clues that I had discovered in researching indicators of art value include the artist, the materials, dimensions of the artwork, origin and transactional history of the artwork. There is some indication that factors including the number of times a piece is exhibited increases its value. This is useful in starting to work with these variables, so I'll start here. Ill first do some cleaning and changing of the dimensions of the artwork. 

In [ ]:
train$height_cm <- as.numeric(train$height_cm)
train$width_cm <- as.numeric(train$width_cm)
test$height_cm <- as.numeric(test$height_cm)
test$width_cm <- as.numeric(test$width_cm)

# changing N/A's to median in height/ weight columns
train$height_cm[is.na(train$height_cm)] <- mean(train$height_cm, na.rm = T)
train$width_cm[is.na(train$width_cm)] <- mean(train$width_cm, na.rm = T)
train$dimensions<-(train$height_cm * train$width_cm)

# same for test
test$height_cm[is.na(test$height_cm)] <- mean(test$height_cm, na.rm = T)
test$width_cm[is.na(test$width_cm)] <- mean(test$width_cm, na.rm = T)
test$dimensions<-(test$height_cm * test$width_cm)

In [ ]:
any(is.na(train$dimensions))

Great! So now we have two new variables: "top selling artists" and "dimensions". There are a few other columns I am interested in learning more about/ sorting through for a useful data set. I couldn't find anything useful as to whether or not the currency an artwork is originally sold in has an impact on its selling value. Lets just take a quick peek as to whether or not there is any useful clues within the data. 

In [ ]:
train$original_currency<- as.factor(train$original_currency)

# organize currency by selling value
avg_sales_by_currency <- train %>%
  group_by(original_currency) %>%
  summarise(avg_sales = mean(price_realized_usd))

# bar chart of currency values
ggplot(avg_sales_by_currency, aes(x = original_currency, y = avg_sales, fill = original_currency)) +
  geom_bar(stat = "identity", position = "dodge", color = "white") +
  labs(title = "Average Sales by Currency Type",
       x = "Currency Type",
       y = "Average Sales (USD)") +
  scale_fill_manual(values = c("USD" = "blue", "GBP" = "green", "HKD" = "red", "CHF" = "yellow", "EUR" = "orange", "CNY" = "purple", "INR" = "black")) +
  theme_minimal()

Some indication here that paintings original currency in USD, CNY, and HKD turn out to be the top selling paintings. Because we are preparing for regularization methods, one-hot encoding may work best for these variables. I will keep the original factor column in the data and experiment with that as well. 

In [ ]:
# selecting by acquired from artist
train <- train %>%
  mutate(source = case_when(
    grepl("acquired.*the artist", provenance, ignore.case = TRUE) ~ "TRUE",
    TRUE ~ "FALSE"
  ))

# selecting by oil painting
train %>% group_by(source) %>% summarise(mean(price_realized_usd))
train <- train %>%
  mutate(oil_painting = case_when(
    grepl("\\boil\\b", details, ignore.case = TRUE) ~ "TRUE",
    TRUE ~ "FALSE"
  ))

# checking mean sales for oil paintings
train %>% group_by(oil_painting) %>% summarise(mean(price_realized_usd))

# same for test
test<- test %>%
  mutate(source = case_when(
    grepl("acquired.*the artist", provenance, ignore.case = TRUE) ~ "TRUE",
    TRUE ~ "FALSE"
  ))

test <- test %>%
  mutate(oil_painting = case_when(
    grepl("\\boil\\b", details, ignore.case = TRUE) ~ "TRUE",
    TRUE ~ "FALSE"
  ))


In researching art and its selling value, it was noted that oil paintings are sometimes valued more. This seemes to be for mixed factors including cost of materials, labour times, and the percieved skill required to make oil paintings. It looks like this may be a factor related to sales (unclear if causal). There seems to be higher average sales for oil paintings than others. This will also be used in the final dataset. 

In [ ]:
plot(train$estimate_high_usd, train$price_realized_usd)
plot(train$estimate_low_usd, train$price_realized_usd)


Both the estimates show a positive correlation with the actual selling point. This means that even on the lower end of the estimated sale price, the higher the estimated sale value indicates the artwork likely sold for a higher price. That being said, there seem to be a lot of 0's or no estimates in these rows. Either because the data is missing, or there was no assumed estimate for those observations. It looks like these 0's represent a marginal amount of the total observations, so lets clean these columns a bit. 

In [ ]:
set.seed(123)

# Identify zero values
zero_indices <- which(train$estimate_high_usd == 0)

# Generate random values within the range of the variable
random_values <- runif(length(zero_indices), min(train$estimate_high_usd), max(train$estimate_high_usd))

# Replace zeros with random values
train$estimate_high_usd[zero_indices] <- random_values

# Same for low estimate
zero_indices <- which(train$estimate_low_usd == 0)
random_values <- runif(length(zero_indices), min(train$estimate_low_usd), max(train$estimate_low_usd))
train$estimate_low_usd[zero_indices] <- random_values

# test
zero_indices <- which(test$estimate_high_usd == 0)
random_values1<- runif(length(zero_indices), min(test$estimate_high_usd), max(test$estimate_high_usd))
test$estimate_high_usd[zero_indices] <- random_values1
zero_indices1<- which(test$estimate_low_usd == 0)
random_values1<- runif(length(zero_indices1), min(test$estimate_low_usd), max(test$estimate_low_usd))
test$estimate_low_usd[zero_indices1]<-random_values1

In [ ]:
plot(train$estimate_high_usd, train$price_realized_usd)
plot(train$estimate_low_usd, train$price_realized_usd)


For now, I have just assigned the 0's random values throughout the data. It seems like there are some outliers. A bit more research into the outliers may indicate which artist these values belong to, with some details as to why they sold for so high. It may be that they were values sold directly from the artist, and therefore not put on auction to be given estimates. Because we are using regularization regressions, we will leave them as is for now. It may be the case that looking into these 0's and managing them a bit differently could improve the model. Or, it may just reveal some other useful information. For now, we will consider it as is. 

In [ ]:
# selecting based on currency
train <- train %>%
  mutate(currency = case_when(
    original_currency %in% c("USD", "CNY") ~ TRUE,
    TRUE ~ FALSE
  ))

# same for test
test <- test %>%
  mutate(currency = case_when(
    original_currency %in% c("USD", "CNY") ~ TRUE,
    TRUE ~ FALSE
  ))


In [ ]:
# cleaned data
clean_train<-train %>% select(-c(object_id, details, category, artist, height_cm, width_cm, auction, location, details, provenance, literature, exhibited, date, original_currency))
clean_test<- test %>% select(-c(object_id, details, category, artist, height_cm, width_cm, auction, location, details, provenance, literature, exhibited, date, original_currency))

In [ ]:
head(clean_train)
head(clean_test)

In [ ]:
clean_train <- clean_train %>%
  mutate(currency = as.character(currency))

clean_test <- clean_test %>%
  mutate(currency = as.character(currency))

# Ridge Regression

Looking into this problem, it seems that the normal rmsle() function has difficulty handling negative values: https://www.rdocumentation.org/packages/Metrics/versions/0.1.4/topics/rmsle. This is tough because ridge can produce negative values. One solution is to change these values to non-negative, though that does introduce some bias. Thanks Molly for the code!

In [ ]:
x <- model.matrix(price_realized_usd ~., clean_train)[,-3]
y <- clean_train$price_realized_usd

In [ ]:
# Split data into training and testing sets
set.seed(123)
train_idx<-sample(1:nrow(clean_train), nrow(clean_train) / 2)
test_idx<-setdiff(1:nrow(clean_train), train_idx)
y.test<-clean_train$price_realized_usd[test_idx]

In [ ]:
grid<- 10^seq(-2, 2, length = 100)

In [ ]:
set.seed(123)
ridge.mod <- glmnet(x[train_idx,], y[train_idx], alpha = 0, lambda = grid , thresh = 1e-12)
cv.out <- cv.glmnet(x[train_idx,], y[train_idx], alpha = 0)
bestlam <- cv.out$lambda.min
ridge_pred <- predict(ridge.mod , s = bestlam, newx = x[test_idx, ])

ridge_pred<- pmax(ridge_pred, 0)
y.test <-pmax(y.test, 0)

rmsle_value <- rmsle(ridge_pred, y.test)
cat("RMSLE:", rmsle_value, "\n")

Wow, what a journey that was! As with everyone else, it seemed that the problem with this code was determing the best approach for calculating rmsle. I think the issue folks were having is that the rmsle function is incompatible with 0 or negative values. See the attached document: https://www.rdocumentation.org/packages/Metrics/versions/0.1.4/topics/rmsle In order to make it work, I had to manually change the values to non-negative. Though this can introduct some bias, it worked great generating RSMLE. The result is an RMSLE of 1.65. 

# Lasso Regression

In [ ]:
set.seed(123)

# Train lasso regression model with cross-validation
cv_model <- cv.glmnet(x[train_idx,],
  y[train_idx],
  alpha = 1,  # 
  nfolds = 5 
)

# Obtain the best lambda value
best_lambda <- cv_model$lambda.min

# Train lasso regression model with the best lambda on the entire training set
lasso_model <- glmnet(x[train_idx,], y[train_idx], alpha = 1, lambda = best_lambda)

predicted_values <- predict(lasso_model, newx = x[test_idx,])

ridge_pred<- pmax(predicted_values, 0)
y_test <-pmax(y.test, 0)


rmsle_value <- rmsle(predicted_values, y.test)
cat("RMSLE:", rmsle_value, "\n")

I used the same RMSLE formula and checked it with my original approach to make sure there wouldnt be much of a difference. Looks good. 

# Random Forest

In [ ]:
# Set up train control
trControl <- trainControl(method = "cv",
                          number = 5,
                          verboseIter = FALSE)

tuneGrid <- expand.grid(.mtry = c(1:6))

store_maxtrees <- list()


for (ntree in c(200, 400, 500)) {
  set.seed(123)
  rf_maxtrees <- train(
    price_realized_usd ~ .,
    data = clean_train[train_idx, ],
    method = "rf",
    metric = "RMSE",
    tuneGrid = tuneGrid,
    trControl = trControl,
    nodesize = 14,
    maxnodes = 24,
    num.trees = ntree,
    importance = TRUE, 
  )
  key <- as.character(ntree)
  store_maxtrees[[key]] <- rf_maxtrees
}

# Combine the results
results_tree <- resamples(store_maxtrees)

# Summary of the results
summary(results_tree)


Wow! Also not an easy process. I found this useful tutorial: https://www.guru99.com/r-random-forest-tutorial.html for how to use cross validation with random forests method = "rf" looping through different values of ntree. While this approach was easy to follow, it was taking a long, long time to run under method = "rf". Be patient, it does eventually work!

In [ ]:
# Find the optimal number of trees based on median RMSE
optimal_ntree_index <- which.min(sapply(store_maxtrees, function(model) min(model$results$RMSE)))

# Retrieve the corresponding optimal number of trees
optimal_ntree <- as.numeric(names(store_maxtrees)[optimal_ntree_index])

# Print the optimal number of trees
cat("Optimal Number of Trees:", optimal_ntree, "\n")

In [ ]:
rf_maxtrees$bestTune$num.trees <- optimal_ntree

predictions <- predict(rf_maxtrees, newdata = clean_train[test_idx, ])

rmsle_value <- rmsle(predictions, y.test)
cat("RMSLE:", rmsle_value, "\n")

Random forests 200 shows to be the best fitting model with rmsle 1.44

In [ ]:
final_rf<- train(
  price_realized_usd ~ .,
  data = clean_train[train_idx, ],
  method = "rf",
  metric = "RMSE",
  tuneGrid = tuneGrid,
  trControl = trControl,
  nodesize = 14,
  maxnodes = 24,
  num.trees = 200,  
  importance = TRUE
)

It looks like a tree size of 400 performs the best as determined through cross validation. 

In [ ]:
test_pred <- predict(final_rf, newdata = clean_test)

submission <- data.frame("object_id" = clean_test$object_id, "price_realized_usd" = test_pred)

write_csv(submission,'submission.csv')